# Generate Prsn Thunderbird Demo

In [1]:
from birdy import WPSClient
import requests
from pkg_resources import resource_filename
from bs4 import BeautifulSoup

In [2]:
import os 
if os.path.basename(os.getcwd()) != "thunderbird": # Ensure current directory is always 'thunderbird' for 'resource_filename' to work
    os.chdir('../')

In [3]:
# Set up wps application
url = 'http://localhost:5001/wps'
thunderbird = WPSClient(url=url)

In [4]:
# Check info on `generate_prsn` process
thunderbird.generate_prsn?

Signature:
thunderbird.generate_prsn(
    prec,
    tasmin=None,
    tasmax=None,
    dry_run=None,
    chunk_size=100,
    output_file='None',
    loglevel='INFO',
)
Docstring:
Generate precipitation as snow file from precipitation and minimum/maximum temperature data

Parameters
----------
prec : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Precipitation file to process
tasmin : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmin file to process
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmax file to process
chunk_size : integer
    Number of time slices to be read/written at a time
output_file : string
    Optional custom name of output file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
dry_run : boolean
    Checks file to ensure compatible with process

Returns
-------
output : ComplexData:mimetype:`applic

In [5]:
# Test local netcdf files
pr_file_local = resource_filename("tests", "data/pr_week_test.nc")
tasmin_file_local = resource_filename("tests", "data/tasmin_week_test.nc")
tasmax_file_local = resource_filename("tests", "data/tasmax_week_test.nc")

dry_output = thunderbird.generate_prsn(pr_file_local, tasmin_file_local, tasmax_file_local, chunk_size=50, dry_run=True)

In [6]:
# Process dry run output
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: /tmp/pywps_process_h0gti66d/pr_week_test.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['pr']
File: /tmp/pywps_process_h0gti66d/tasmin_week_test.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmin']
File: /tmp/pywps_process_h0gti66d/tasmax_week_test.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmax']



In [7]:
# generate prsn
output = thunderbird.generate_prsn(pr_file_local, tasmin_file_local, tasmax_file_local, chunk_size=50, dry_run=False)

In [8]:
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl

<metaurl mediatype="application/x-netcdf">
     http://localhost:5001/outputs/16b37881-b726-11ea-afd0-c86000e3f2fd/prsn_day_BCCAQv2_NorESM1-M_historical+rcp26_r1i1p1_19500101-19500107_Canada.nc
    </metaurl>

In [9]:
# Custom output filename
output = thunderbird.generate_prsn(pr_file_local, tasmin_file_local, tasmax_file_local, chunk_size=50, dry_run=False, output_file="prsn_test_local.nc")

In [10]:
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl

<metaurl mediatype="application/x-netcdf">
     http://localhost:5001/outputs/16f9e6b3-b726-11ea-afd0-c86000e3f2fd/prsn_test_local.nc
    </metaurl>

In [11]:
# Test opendap netcdf files
pr_file_opendap = "http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/pr_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"
tasmin_file_opendap = "http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"
tasmax_file_opendap = "http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"

dry_output = thunderbird.generate_prsn(pr_file_opendap, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=True, output_file="prsn_test_opendap.nc")

In [12]:
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/pr_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['pr']
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmin']
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmax']



In [13]:
output = thunderbird.generate_prsn(pr_file_opendap, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=False, output_file="prsn_test_opendap.nc")

In [14]:
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl

<metaurl mediatype="application/x-netcdf">
     http://localhost:5001/outputs/1d6680f0-b726-11ea-afd0-c86000e3f2fd/prsn_test_opendap.nc
    </metaurl>

In [15]:
# Test netcdf files from both sources
dry_output = thunderbird.generate_prsn(pr_file_local, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=True, output_file="prsn_test_mixed.nc")

In [16]:
req = requests.get(dry_output.get()[0])
print(req.content.decode('utf-8'))

Dry Run
File: /tmp/pywps_process_ifr8pnxm/pr_week_test.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['pr']
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmin']
File: http://docker-dev03.pcic.uvic.ca:8083/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc
project: CMIP5
model: NorESM1-M
institute: PCIC
experiment: historical,rcp26
ensemble_member: r1i1p1
dependent_varnames: ['tasmax']



In [17]:
output = thunderbird.generate_prsn(pr_file_local, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=False)

In [18]:
req = requests.get(output.get()[0])
BeautifulSoup(BeautifulSoup(req.content.decode('utf-8')).prettify()).metaurl

<metaurl mediatype="application/x-netcdf">
     http://localhost:5001/outputs/20e800b4-b726-11ea-afd0-c86000e3f2fd/prsn_day_BCCAQv2_NorESM1-M_historical+rcp26_r1i1p1_19500101-19500107_Canada.nc
    </metaurl>